In [10]:
import sys
path = "C:\\Users\\Paul Valcke\\Documents\\GitHub\\GEMMES"
sys.path.insert(0, path )
import pygemmes as pgm

# Ipywidgets relative 
import ipywidgets as widgets
from IPython.display import display,HTML,clear_output,Markdown
import numpy as np
import pandas as pd
from IPython.display import IFrame
import threading
# Options
default = '__EMPTY__' ## DEFAULT MODEL LOAD IN THE SYSTEM 
style = {'description_width': 'initial'}
%matplotlib widget
pd.set_option('display.max_colwidth', None)
AllModels = pgm.get_available_models(details=True)
FULL = pgm.get_available_models(FULL=True)
hub=pgm.Hub(default,verb=False)
hub.run(verb=False)
Categories = ['eqtype','source_exp','definition','com','group','units','symbol','isneeded']

with open(path+'\\README.md', 'r') as fh: tutorial = fh.read()
with open(path+'\\Tutorial-model.md', 'r') as fh: tutorialmodel = fh.read()

def pprint(ldf):
    ''' 
    Print with newline in dataframe
    '''
    try :
        ldf = ldf.style.set_properties(**{'text-align': 'left'})
    except BaseException:
        pass
    return display(HTML(ldf.to_html().replace("\\n","<br>")))

In [46]:
# TAB EXPLORATION / LOAD 
def get_Exploration0(hub):
    '''General tutorial on pygemmes'''
    for _ in range(1): # first line : description 
        ### Widget declaration 
        ShowPygemmes  = widgets.Button(description='Pygemmes description',style=style)
        ShowModels    = widgets.Button(description='Model list',style=style)
        ShowFields    = widgets.Button(description='Fields list',style=style)
        ShowFunctions = widgets.Button(description='Functions list',style=style)
        ShowPlots     = widgets.Button(description='Plots list',style=style)
        ShowUse       = widgets.Button(description='Use Tutorial',style=style)
        ShowWriting   = widgets.Button(description='Model Tutorial',style=style)
        ShowClear     = widgets.Button(description='Clear',style=style)
        ShowOut = widgets.Output()

        ### Widget Functions 
        def ShowPygemmes_event(obj):
            with ShowOut:
                ShowOut.clear_output()
                print(pgm.__doc__)
        ShowPygemmes.on_click(ShowPygemmes_event)
        def ShowModels_event(obj):
            with ShowOut:
                ShowOut.clear_output()
                pprint(pgm.get_available_models(details=True))
        ShowModels.on_click(ShowModels_event)
        def ShowFields_event(obj):
            with ShowOut:
                ShowOut.clear_output()
                pprint(pgm.get_available_fields(exploreModels=True))
        ShowFields.on_click(ShowFields_event)
        def ShowFunctions_event(obj):
            with ShowOut:
                ShowOut.clear_output()
                pprint(pgm.get_available_functions())
        ShowFunctions.on_click(ShowFunctions_event)
        def ShowPlots_event(obj):
            with ShowOut:
                ShowOut.clear_output()
                pprint(pgm.get_available_plots())
        ShowPlots.on_click(ShowPlots_event)
        def ShowClear_event(obj):
            with ShowOut:
                ShowOut.clear_output()
        ShowClear.on_click(ShowClear_event)
        def ShowUse_event(obj):
            with ShowOut:
                ShowOut.clear_output()
                display(Markdown(tutorial))
        ShowUse.on_click(ShowUse_event)
        def ShowWriting_event(obj):
            with ShowOut:
                ShowOut.clear_output()
                display(Markdown(tutorialmodel))
        ShowWriting.on_click(ShowWriting_event)
        
    return widgets.VBox([widgets.HBox([ShowPygemmes,ShowModels,ShowFields,ShowFunctions,ShowPlots,ShowUse,ShowWriting]),
                                    ShowOut])

def get_Loading(hub):
    '''Loading model'''
    ### Widget declaration  
    Loaddropdown = widgets.Dropdown(options=list(AllModels.index),value=hub.dmodel['name'],description='Model file :')
    Loadpresets = widgets.Dropdown(options=[None]+AllModels.loc[hub.dmodel['name']].loc['Preset'], description='Preset :')
    Loadout = widgets.Output()
    createhub = widgets.Button(description='LOAD !')

    def valuechange(change):
        Loadpresets.options = [None]+AllModels.loc[change['new']].loc['Preset']
        with Loadout:
            Loadout.clear_output()
            display(Markdown('# Model: '+FULL.loc[change['new']].loc['name']))
            display(Markdown(FULL.loc[change['new']].loc['description']))
            display(Markdown(FULL.loc[change['new']].loc['address']))
            if FULL.loc[change['new']].loc['description'] != FULL.loc[change['new']].loc['longDescription']:
                display(Markdown(FULL.loc[change['new']].loc['longDescription']))
            display(Markdown('# Presets'))
            display(pd.DataFrame(pd.DataFrame(FULL.loc[change['new']].loc['presets']).transpose()['com']))
            display(Markdown('# Supplements'))
            display(FULL.loc[change['new']].loc['supplements'])
    Loaddropdown.observe(valuechange, names='value')

    def createhub_event(obj):
        global hub
        with Loadout:
            hub= pgm.Hub(Loaddropdown.value,preset=Loadpresets.value,verb=True)
    createhub.on_click(createhub_event)
    return widgets.VBox([widgets.HBox([Loaddropdown,Loadpresets,createhub]),Loadout])

def set_value(hub):     
    vardrop = widgets.Dropdown(options=list(set(hub.dmisc['dfunc_order']['parameter']
                        +hub.dmisc['dfunc_order']['parameters']
                        +hub.dmisc['dfunc_order']['differential'])
                        -set(['__ONE__','dt'])),value=list(set(hub.dmisc['dfunc_order']['parameter']
                        +hub.dmisc['dfunc_order']['parameters']
                        +hub.dmisc['dfunc_order']['differential'])
                        -set(['__ONE__','dt']))[0],description='Field to change :',style=style)

    Parrallel2 = widgets.Dropdown(
        options=hub.dparam['nx']['list'],
        value=hub.dparam['nx']['list'][0],
        description='parrallel:',
        continuous_update=True
    ) 
    Region2 = widgets.Dropdown(
        options=hub.dparam['nr']['list'],
        value=hub.dparam['nr']['list'][0],
        description='Region:',
        continuous_update=True
    ) 
    Multi1 = widgets.Dropdown(
        options=hub.dparam[hub.dparam[vardrop.value]['size'][0]]['list'],
        value=hub.dparam[hub.dparam[vardrop.value]['size'][0]]['list'][0],
        description='Multi1:',
        continuous_update=True
    ) 
    Multi2 = widgets.Dropdown(
        options=hub.dparam[hub.dparam[vardrop.value]['size'][1]]['list'],
        value=hub.dparam[hub.dparam[vardrop.value]['size'][1]]['list'][0],
        description='Multi2:',
        continuous_update=True
    ) 

    def update_multi1(*args):
        Multi1.options = hub.dparam[hub.dparam[vardrop.value]['size'][0]]['list']
    def update_multi2(*args):
        Multi2.options = hub.dparam[hub.dparam[vardrop.value]['size'][1]]['list']
    vardrop.observe(update_multi1, 'value')
    vardrop.observe(update_multi2, 'value')

    def setval(vardrop,Parrallel2,Region2,Multi1,Multi2):
        print(vardrop.value,Parrallel2.value,Region2.value,Multi1.value,Multi2.value)
        IndP = hub.dparam['nx']['list'].index(Parrallel2.value)
        IndR = hub.dparam['nr']['list'].index(Region2.value)
        IndM1= hub.dparam[hub.dparam[vardrop.value]['size'][0]]['list'].index(Multi1.value)
        IndM2= hub.dparam[hub.dparam[vardrop.value]['size'][1]]['list'].index(Multi2.value)
        print(IndP,IndR,IndM1,IndM2)
        val = str(hub.dparam[vardrop.value]['value'][0,IndP,IndR,IndM1,IndM2]) if vardrop.value in hub.dmisc['dfunc_order']['differential'] else (str(hub.dparam[vardrop.value]['value'][  IndP,IndR,IndM1,IndM2]) if hub.dparam[vardrop.value]['group']!='Numerical' 
            else  str(hub.dparam[vardrop.value]['value']))
        print("Val"+val)
        print()
        return val

    setvalue =  widgets.FloatText(
        value=  setval(vardrop,Parrallel2,Region2,Multi1,Multi2) ,
        description='Set value:',
    ) 
    def update_val(*args): setvalue.value = setval(vardrop,Parrallel2,Region2,Multi1,Multi2)
    vardrop.observe(update_val,'value')
    Multi1.observe(update_val,'value')
    Multi2.observe(update_val,'value')


    clickset = widgets.Button(description='Set !')
    def clickset_event(obj):
        #hub.set_dparam(vardrop.value,{'nr':[Region2],
        #                    'nx':Parrallel2,
        #                    'first': Multi1,
        #                    'second': Multi2,
        #                    'value':float(setvalue.value)} )
        hub.set_dparam(vardrop.value,[Parrallel2,Region2, Multi1,Multi2,float(setvalue.value)] )
    clickset.on_click(clickset_event)           

    return widgets.VBox([widgets.HBox([vardrop,setvalue,clickset]),Parrallel2,Region2,Multi1,Multi2,])

# Network
def get_network(hub):
    createNetwork = widgets.Button(description='Create Network')
    dropdownP = widgets.Dropdown(options=[True,False],value=True,description='Show parameters :',style=style)
    dropdownA = widgets.Dropdown(options=[True,False],value=True,description='Show Auxilliary :',style=style)
    outnet= widgets.Output()

    def createNetwork_event(obj):
        hub.get_Network(params=dropdownP.value,auxilliary=dropdownA)  
        #f=open(r'C:\Users\Paul Valcke\Documents\GitHub\GEMMES\doc\Goodwin.html')
        #with Loadout:
        #    IFrame(f.read(),width=800,height=800)
    createNetwork.on_click(createNetwork_event)

    viewFields = widgets.Button(description='view Fields')
    viewValues = widgets.Button(description='view Values')

    return widgets.HBox([dropdownP,dropdownA,createNetwork])

# Type Summary 
def get_introspect(hub): 
    clicksummary = widgets.Button(description='get summary')
    clickFields =widgets.Button(description='get Fields')
    dropdownCAT = widgets.Dropdown(options=Categories,value='eqtype',description='Sorting :')
    outINTROS = widgets.Output()

    def clicksummary_event(obj):
        with outINTROS:
            
            outINTROS.clear_output()
            hub.get_summary()
    clicksummary.on_click(clicksummary_event)

    def clickFields_event(obj):
        with outINTROS:
            outINTROS.clear_output()
            R=hub.get_dparam()
            Rpandas= {k0:{k:R[k0][k] for k,v in R[k0].items() if k in Categories} for k0 in R.keys()}
            AllFields=pd.DataFrame(Rpandas,index=Categories).transpose().sort_values(by=[dropdownCAT.value])
            pprint(AllFields.replace(np.nan,''))
    clickFields.on_click(clickFields_event)

    return widgets.VBox([widgets.HBox([dropdownCAT,clickFields]),outINTROS])

# Values summary
def get_value(hub):
        TimeSlider = widgets.FloatRangeSlider(
            value=[hub.dparam['time']['value'][0,0,0,0,0], hub.dparam['time']['value'][-1,0,0,0,0]],
            min=hub.dparam['time']['value'][0,0,0,0,0],
            max=hub.dparam['time']['value'][-1,0,0,0,0],
            step=hub.dparam['time']['value'][1,0,0,0,0]-hub.dparam['time']['value'][0,0,0,0,0],
            description='Time range:',
            disabled=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            continuous_update=True
        ) 
        Parrallel = widgets.Dropdown(
            options=np.arange(hub.dparam['nx']['value']),
            value=0,
            description='parrallel :',
            continuous_update=True
        ) 
        Region = widgets.Dropdown(
            options=hub.dparam['nr']['list'],
            value=hub.dparam['nr']['list'][0],
            description='Region :',
            continuous_update=True
        ) 
        clickValues = widgets.Button(description='get values')
        outValues = widgets.Output()
        def clickValues_event(obj):
            with outValues:
                outValues.clear_output()
                hub.get_summary()
        clickValues.on_click(clickValues_event)

        return widgets.VBox([widgets.HBox([TimeSlider,Parrallel,Region,clickValues]), outValues])

# RUN 
def get_run(hub):
    clickrun = widgets.Button(description='RUN!')
    outrun = widgets.Output()
    def clickrun_event(obj):
        global hub
        with outrun:
            hub.run()
    clickrun.on_click(clickrun_event)

    setReinterp =  widgets.FloatText(
    value=hub.dparam['nt']['value'],
    description='Reinterpolate on:',style=style)

    #  Set  
    clickset = widgets.Button(description='Set !')
    def clickset_event(obj):
        hub.reinterpolate_dparam(int(setReinterp.value))
        with outrun:
            print("reinterpolated")
    clickset.on_click(clickset_event)

    return widgets.VBox([clickrun,widgets.HBox([setReinterp,clickset]),outrun])

# TAB PLOTS
def get_plot(hub):
    ########### PRESETS ############## 
    Outplot=widgets.Output()
    B1 = widgets.Button(description='Presets',style=style)
    def B1e(obj): 
        with Outplot: 
            Outplot.clear_output(); 
            hub.plot_preset()
    B1.on_click(B1e)

    ########### ONE VARIABLE ########
    def PLOToneVariable(hub):
            TimeSlider = widgets.FloatRangeSlider(
                    value=[hub.dparam['time']['value'][0,0,0,0,0], hub.dparam['time']['value'][-1,0,0,0,0]],
                    min=hub.dparam['time']['value'][0,0,0,0,0],
                    max=hub.dparam['time']['value'][-1,0,0,0,0],
                    step=hub.dparam['time']['value'][1,0,0,0,0]-hub.dparam['time']['value'][0,0,0,0,0],
                    description='Time range:',
                    disabled=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.1f',
                    continuous_update=True
                ) 
            Parrallel = widgets.Dropdown(
                options=np.arange(hub.dparam['nx']['value']),
                value=0,
                description='parrallel :',
                continuous_update=True
            ) 
            Region = widgets.Dropdown(
                options=hub.dparam['nr']['list'],
                value=hub.dparam['nr']['list'][0],
                description='Region :',
                continuous_update=True
            ) 
            outPLOT = widgets.Output()
            Plotvars = hub.dmisc['dfunc_order']['statevar']+hub.dmisc['dfunc_order']['differential']
            varXdrop = widgets.Dropdown(options=Plotvars,value='time',description='X axis :')
            varYdrop = widgets.Dropdown(options=Plotvars,value='time',description='Y axis :')
            varCdrop = widgets.Dropdown(options=Plotvars,value='time',description='color :')

            clickplot = widgets.Button(description='Plot !')
            def clickplot_event(obj):
                with outPLOT:
                    pgm.plots.phasespace(hub,x=varXdrop.value,
                                    y=varYdrop.value,
                                    color = varCdrop.value,idx=0,Region=0)
            clickplot.on_click(clickplot_event)

            clickclear = widgets.Button(description='clear')
            def clickclear_event(obj):
                with outPLOT:
                    #plt.close('all')
                    outPLOT.clear_output()
            clickclear.on_click(clickclear_event)

            return widgets.VBox([widgets.HBox([TimeSlider,Parrallel,Region]),
                                    widgets.HBox([varXdrop,varYdrop,varCdrop]),
                                    widgets.HBox([clickplot,clickclear]),outPLOT])    
    B2 = widgets.Button(description='One Variable',style=style)
    def B2e(obj): 
        with Outplot: 
            Outplot.clear_output(); 
            display(Markdown('## 1 variable'))
            display(PLOToneVariable(hub))
    B2.on_click(B2e)

    B21= widgets.Button(description='Sensitivity',style=style)
    B22= widgets.Button(description='3D Plot',style=style)
    B3 = widgets.Button(description='By Units',style=style)
    B4 = widgets.Button(description='N Y axes',style=style)
    B5 = widgets.Button(description='Cycles',style=style)
    B6 = widgets.Button(description='Repartition',style=style)
    B7 = widgets.Button(description='Sankey',style=style)

    return widgets.VBox([widgets.HBox([B1,B2,B21,B22]),
                            widgets.HBox([B3,B4,B5,B6,B7]),
                            Outplot]) 


#########################################################################
#                      INTERFACE TO RULE THEM ALL  
# #######################################################################
for _ in range(1):
    Bnames = ['Pygemmes exploration','Loading','Set Values','Run','Explore','Plot']
    Buttons = {b : widgets.Button(description=b,style=style) for b in Bnames }
    OUT = widgets.Output()

    ### Widget Functions 
    def pgm_event(obj): 
        with OUT: 
            OUT.clear_output(); 
            display(Markdown('# Exploring Pygemmes content and use'))
            display(get_Exploration0(hub))
    Buttons['Pygemmes exploration'].on_click( pgm_event)
    def load_event(obj): 
        with OUT: 
            OUT.clear_output(); 
            display(Markdown('# Loading model'))
            display(get_Loading(hub))
    Buttons['Loading'].on_click( load_event)
    def a_event(obj): 
        with OUT: 
            OUT.clear_output(); 
            display(Markdown('# Changing values'))
            display(set_value(hub))
    Buttons['Set Values'].on_click( a_event)
    def b_event(obj): 
        with OUT: 
            OUT.clear_output()
            display(Markdown('# Run the simulation'))
            display(get_run(hub))
    Buttons['Run'].on_click( b_event)
    def c_event(obj): 
        with OUT: 
            OUT.clear_output()
            display(Markdown('# Network Representation'))
            display(get_network(hub)) 
            display(Markdown('# Explore fields type'))
            display(get_introspect(hub))
            display(Markdown('# Explore fields values'))
            display(get_value(hub))
    Buttons['Explore'].on_click( c_event)
    def d_event(obj): 
        with OUT: 
            OUT.clear_output() 
            display(Markdown('## Plots'))
            display(get_plot(hub))
    Buttons['Plot'].on_click( d_event)
    display(widgets.VBox([widgets.HBox([B for B in Buttons.values()]),OUT]))



Gamma no parrallel local Consumption Monosectoral


ValueError: 'Monosectoral' is not in list

In [ ]:
for X in ['Pygemmes','Models','Fields','Functions','Plots','Clear','Out']:
    strout =f'''def Show{X}_event(obj):
    with ShowOut:
        ShowOut.clear_output()
        pass
Show{X}.on_click(ShowM{X}_event)'''
    print(strout)

In [ ]:
'''
ShowPygemmes.on_click(ShowPygemmes_event)


ultimateout= widgets.Output()
varXdrop = widgets.Dropdown(options=['Pygemmes exploration','Load model',],value='time',description='X axis :')

display(Exploration0)
display(Load0)
display(Networkline)
display(introspect_widgets)
display(Values_widgets)

display(run_widgets)
'''

In [17]:
R=hub.get_dparam()
R['Nprod']

In [18]:
R['D']['size']

['Nprod', '__ONE__']

In [37]:
hub.dparam['b']

{'value': array([[[[1.],
          [1.],
          [1.]]]]),
 'size': ['Nprod', '__ONE__'],
 'definition': 'part of capital in prod intensity',
 'units': '',
 'group': 'Production',
 'com': 'No comment',
 'symbol': '$b$',
 'multisect': '',
 'inmodel': ['CHIMES0',
  'ECHIMES',
  'Goodwin_CES',
  'Inflation',
  'Markup',
  'MultisectoralAggregates',
  'reduced_GKCES']}

In [44]:
help(hub.set_dparam)

Help on method set_dparam in module pygemmes._core:

set_dparam(key=None, value=None, verb=True, **kwargs) method of pygemmes._core.Hub instance
    Your best friend to change the fields values or sizes in the system.
    It can change :
        * dimensions ( duration of the simulated tine 'Tmax',
                       duration of one timestep 'dt'
                       number of system in parrallel 'nx',
                       number of regions             'nr',
                       number of sectors in a multisector object _nameofthedimension...
                       )
        * values of parameters and initial conditions, either for all systems/regions/sector, either indexwise
    
    if you want to change only one field, you can do "set_dparam(key,values)"
    if you want to change multiple fields at once you can do "set_dparam(**dict)" with dict={key:values},
    the way to put multiple values or specific sector will be explained in 2)
    
    1) ### CHANGE DIMENSIONS ####